In [29]:
import json
from random import randint
import traceback
from copy import deepcopy

data = [
    {"level": 4, "name": "Bob", "forca": 16, "dex": 12, "cons": 12, "armor": 1, "weapon": "Machado Pequeno"},
    {"level": 2, "name": "John", "forca": 12, "dex": 10, "cons": 12, "armor": 2, "weapon": "Espada Curta"},
    {"level": 5, "name": "Aragorn", "forca": 12, "dex": 10, "cons": 10, "armor": 6, "weapon": "Espada Curta"},
    {"level": 1, "name": "Legolas", "forca": 12, "dex": 16, "cons": 10, "armor": 2, "weapon": "Espada Longa"},
    {"level": 3, "name": "Gimli", "forca": 12, "dex": 10, "cons": 10, "armor": 6, "weapon": "Machado Grande"},
    {"level": 6, "name": "Gandalf", "forca": 12, "dex": 12, "cons": 10, "armor": 2, "weapon": "Machado Pequeno"},
    {"level": 1, "name": "Frodo", "forca": 12, "dex": 14, "cons": 10, "armor": 2, "weapon": "Adaga"},
    {"level": 2, "name": "San", "forca": 12, "dex": 14, "cons": 10, "armor": 1, "weapon": "Espada Longa"},
    {"level": 4, "name": "Pipin", "forca": 16, "dex": 12, "cons": 12, "armor": 1, "weapon": "Espada Longa"},
    {"level": 2, "name": "Mery", "forca": 12, "dex": 14, "cons": 12, "armor": 2, "weapon": "Espada Longa"},
    {"level": 1, "name": "Ragnar", "forca": 18, "dex": 10, "cons": 10, "armor": 6, "weapon": "Machado Pequeno"},
    {"level": 3, "name": "Kratos", "forca": 14, "dex": 10, "cons": 14, "armor": 2, "weapon": "Machado Grande"},
    {"level": 3, "name": "Elvis", "forca": 12, "dex": 10, "cons": 10, "armor": 6, "weapon": "Adaga"},
    {"level": 1, "name": "Saitama", "forca": 12, "dex": 10, "cons": 10, "armor": 2, "weapon": "Espada Longa"},
    {"level": 1, "name": "Rob", "forca": 14, "dex": 12, "cons": 10, "armor": 2, "weapon": "Espada Curta"},
    {"level": 5, "name": "Will", "forca": 12, "dex": 10, "cons": 12, "armor": 5, "weapon": "Espada Longa"}
]

In [33]:
def get_weapon(w):
    
    weapon = {
        "Espada Longa": Weapon(1, 8),
        "Espada Curta": Weapon(1, 6),
        "Machado Pequeno": Weapon(1, 8),
        "Machado Grande": Weapon(1, 12),
        "Adaga": Weapon(1, 4),
    }[w]
    
    if (weapon is None):
        raise Exception("Arma não encontrada")
        
    return weapon


def iniciativa(p1, p2):
    v1 = p1.get_iniciativa()
    v2 = p2.get_iniciativa()
    if v1 == v2:
        return iniciativa(p1, p2)
    
    return v1 > v2


def get_bonus(val):
    bonus = 0
    if val == 10 or val == 11:
        return bonus
    
    if val % 2 == 0:
        bonus = (val - 10) / 2
    else:
        bonus = (val - 11) / 2
        
    return bonus


def combate(p_1, p_2):
    p1, p2 = deepcopy(p_1), deepcopy(p_2)
    while p1.pv > 0 and p2.pv > 0:
        if p1.ataque() - p2.defesa() > 0:
            p2.pv -= p1.damage()
            
        if p2.ataque() - p1.defesa() > 0:
            p1.pv -= p2.damage()
            
    if p1.pv > p2.pv:
        return p_1
    else:
        return p_2
    

class Weapon:
    def __init__(self, damage_min, damage_max):
        self.min = damage_min
        self.max = damage_max
        
    def damage(self):
        return randint(self.min, self.max)
    

class Player:
    def __init__(self, level, name, forca, dex, cons, armor, weapon):
        self.level = level
        self.name = name
        self.forca = forca
        self.dex = dex
        self.cons = cons
        self.armor = armor
        self.weapon = get_weapon(weapon)
        self.pv = (level * 10 + level * get_bonus(10) + ((level-1) * randint(level-1, (level-1)*10)))
        
    def ataque(self):
        return randint(1, 20) + get_bonus(self.forca)
    
    def defesa(self):
        return 10 + get_bonus(self.dex) + self.armor
    
    def get_iniciativa(self):
        return randint(1, 20) + get_bonus(self.dex)
    
    def damage(self):
        return self.weapon.damage() + get_bonus(self.forca)
    
    def auto_print(self):
        print("Level: " + str(self.level))
        print("Nome: " + self.name)
        print(f"Forca: {self.forca}, Dex: {self.dex}, Cons: {self.cons}, Armor: {self.armor}, PVs: {self.pv}")

In [34]:
from pprint import pprint

def printPlayer(players):
    print('===============')
    for player in players:
        player.auto_print()
    print('===============')

def get_list_players(arr):
    list_players = []
    for i in arr:
        p = Player(
            level=i['level'],
            name=i['name'],
            forca=i['forca'],
            dex=i['dex'],
            cons=i['cons'],
            armor=i['armor'],
            weapon=i['weapon']
        )
        list_players.append(p)
        i['pv'] = deepcopy(p.pv)
        
    return list_players


def rum(arr_ganhadores, arr):
    for i in range(len(arr)):
        if i % 2 == 0:
            jogador_1 = arr[i]
            jogador_2 = arr[i+1]
            if iniciativa(jogador_1, jogador_2):
                winner = combate(p_1=jogador_1, p_2=jogador_2)
            else:
                winner = combate(p_1=jogador_2, p_2=jogador_1)
            arr_ganhadores.append(winner)

    
    if len(arr_ganhadores) > 1:
        printPlayer(arr_ganhadores)
        print("========================")
        return rum([], arr_ganhadores)
        
    return arr_ganhadores

In [35]:
try:
    lst = get_list_players(data)
    ganhador = rum([], lst)
    printPlayer(ganhador)
except Exception as e:
    traceback.print_exc()
    print(str(e))

Level: 4
Nome: Bob
Forca: 16, Dex: 12, Cons: 12, Armor: 1, PVs: 88
Level: 5
Nome: Aragorn
Forca: 12, Dex: 10, Cons: 10, Armor: 6, PVs: 198
Level: 6
Nome: Gandalf
Forca: 12, Dex: 12, Cons: 10, Armor: 2, PVs: 175
Level: 2
Nome: San
Forca: 12, Dex: 14, Cons: 10, Armor: 1, PVs: 21
Level: 4
Nome: Pipin
Forca: 16, Dex: 12, Cons: 12, Armor: 1, PVs: 121
Level: 3
Nome: Kratos
Forca: 14, Dex: 10, Cons: 14, Armor: 2, PVs: 54
Level: 3
Nome: Elvis
Forca: 12, Dex: 10, Cons: 10, Armor: 6, PVs: 70
Level: 5
Nome: Will
Forca: 12, Dex: 10, Cons: 12, Armor: 5, PVs: 78
Level: 5
Nome: Aragorn
Forca: 12, Dex: 10, Cons: 10, Armor: 6, PVs: 198
Level: 6
Nome: Gandalf
Forca: 12, Dex: 12, Cons: 10, Armor: 2, PVs: 175
Level: 4
Nome: Pipin
Forca: 16, Dex: 12, Cons: 12, Armor: 1, PVs: 121
Level: 5
Nome: Will
Forca: 12, Dex: 10, Cons: 12, Armor: 5, PVs: 78
Level: 5
Nome: Aragorn
Forca: 12, Dex: 10, Cons: 10, Armor: 6, PVs: 198
Level: 4
Nome: Pipin
Forca: 16, Dex: 12, Cons: 12, Armor: 1, PVs: 121
Level: 5
Nome: Aragor

8
3
